<a href="https://colab.research.google.com/github/ngobibibnbe/NLP-transformers/blob/master/Copie_de_FinBert_Model_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook contains demo for our fine-tuned Analyst Tone model. We fine-tuned FinBERT model on 10,000 manually annotated analyst statements. You can use this script and infer sentiment on your customerized dataset.

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Projets/FinBERT

/content/drive/MyDrive/Projets/FinBERT


In [ ]:
! pip3 install -r requirements.txt

     |████████████████████████████████| 133kB 13.2MB/s 
     |████████████████████████████████| 17.3MB 197kB/s 
     |████████████████████████████████| 748.9MB 22kB/s 
     |████████████████████████████████| 8.8MB 49.2MB/s 
     |████████████████████████████████| 5.4MB 49.1MB/s 
     |████████████████████████████████| 11.7MB 186kB/s 
     |████████████████████████████████| 133kB 64.1MB/s 
     |████████████████████████████████| 7.2MB 44.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.23.0 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas

In [ ]:
%load bertModel.py

In [ ]:
import os
import copy
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from bertModel import BertClassification, dense_opt
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
from datasets import text_dataset, financialPhraseBankDataset
import argparse
from sklearn.metrics import f1_score


In [ ]:
labels = {0:'neutral', 1:'positive',2:'negative'}
num_labels= len(labels)
vocab = "finance-uncased"
#vocab_path = "/home/root/new_bert_vocab_uncased_8000"
vocab_path = 'analyst_tone/vocab'
pretrained_weights_path = "analyst_tone/pretrained_weights" # this is pre-trained FinBERT weights
fine_tuned_weight_path = "analyst_tone/fine_tuned.pth"      # this is fine-tuned FinBERT weights
max_seq_length=256
device='cuda:0'

In [ ]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)

/content/drive/My Drive/Projets/FinBERT/bertModel.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.classifier.weight)


In [ ]:
model.load_state_dict(torch.load(fine_tuned_weight_path, map_location='cuda:0'))
model.to(device)


BertClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


# 0 is neutral, 1 is positive, and 2 is negative 

In [ ]:
sentences = ['there is a shortage of capital, and we need extra financing', "growth is strong and we have plenty of liquidity", "there are doubts about our finances", "profits are flat"]

In [ ]:
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

In [ ]:
model.eval()
for sent in sentences: 
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, 256)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, 256)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, 256)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')
    

there is a shortage of capital, and we need extra financing 
FinBERT predicted sentiment:  negative 

growth is strong and we have plenty of liquidity 
FinBERT predicted sentiment:  positive 

there are doubts about our finances 
FinBERT predicted sentiment:  negative 

profits are flat 
FinBERT predicted sentiment:  neutral 



# Naive Bayes model pre-trained on our 10,000 analyst 

---

sentiment dataset

In [ ]:
from joblib import dump, load

In [ ]:
!pip3 install nltk


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
nltk.download('stopwords')

import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
 nltk.download('punkt')
 
pre_proc_X = [preprocess(t) for t in sentences]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
best_model= load('./naive_tfidf.joblib')

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  U

In [ ]:
pred_prob = best_model.predict_proba(pre_proc_X)

In [ ]:
for idx, sent in enumerate(sentences):
    print(sent, '\nNB predicted sentiment: ', labels[np.argmax(pred_prob[idx])], '\n')

there is a shortage of capital, and we need extra financing 
NB predicted sentiment:  negative 

growth is strong and we have plenty of liquidity 
NB predicted sentiment:  positive 

there are doubts about our finances 
NB predicted sentiment:  positive 

profits are flat 
NB predicted sentiment:  positive 

